In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
#tokenizer = AutoTokenizer.from_pretrained("eachadea/vicuna-13b-1.1")
#model = AutoModelForCausalLM.from_pretrained("eachadea/vicuna-13b-1.1",device_map = 'auto')
#tokenizer = AutoTokenizer.from_pretrained("elinas/vicuna-13b-4bit")
#model = AutoModelForCausalLM.from_pretrained("elinas/vicuna-13b-4bit")
tokenizer = AutoTokenizer.from_pretrained("AlekseyKorshuk/vicuna-7b")
model = AutoModelForCausalLM.from_pretrained("AlekseyKorshuk/vicuna-7b")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [2]:
import numpy as np
import torch
def calc_vec(input_text, model, tokenizer):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, output_hidden_states=True)

    #list形式でtensorが入っているoutputsを1次元のnumpyに変換
    vec_list=[]
    for v in outputs[-1]:
        mean_v=v.mean(axis=1)
        vec_list.append(mean_v.numpy().flatten())

    vec=np.array(vec_list).flatten()

    return vec

In [4]:
from tqdm import tqdm
import numpy as np
text_list=[
"吾輩は猫である",
"汝の名は女だ",
"I am a cat",
"ピペットで試薬を三回分取した",
"アセトンは有機溶媒である",
"非プロトン性溶媒として､THF､トルエンなどが挙げられる",
"トルエンをこぼしてしまいました",
"私は猫です",
"私は犬です",
"日本的霊性は，鎌倉時代に禅と浄土系思想によって初めて明白に顕現し，その霊性的自覚が現在に及ぶと述べる．",
]


vec_list=[calc_vec(i,model,tokenizer) for i in tqdm(text_list)]


target_vec=vec_list[0]


from scipy import spatial
def cos_sim(a, b):
    return 1 - spatial.distance.cosine(a, b)


cos_sim_list=[cos_sim(target_vec,i) for i in vec_list]



100%|██████████| 10/10 [00:21<00:00,  2.15s/it]

1.000 吾輩は猫である
0.988 私は猫です
0.986 私は犬です
0.984 汝の名は女だ
0.964 I am a cat
0.960 アセトンは有機溶媒である
0.959 トルエンをこぼしてしまいました
0.947 ピペットで試薬を三回分取した
0.871 非プロトン性溶媒として､THF､トルエンなどが挙げられる
0.654 日本的霊性は，鎌倉時代に禅と浄土系思想によって初めて明白に顕現し，その霊性的自覚が現在に及ぶと述べる．


In [5]:
for i in np.argsort(cos_sim_list)[::-1]:
    print("{:.3f}".format(cos_sim_list[i]),text_list[i])

1.000 吾輩は猫である
0.988 私は猫です
0.986 私は犬です
0.984 汝の名は女だ
0.964 I am a cat
0.960 アセトンは有機溶媒である
0.959 トルエンをこぼしてしまいました
0.947 ピペットで試薬を三回分取した
0.871 非プロトン性溶媒として､THF､トルエンなどが挙げられる
0.654 日本的霊性は，鎌倉時代に禅と浄土系思想によって初めて明白に顕現し，その霊性的自覚が現在に及ぶと述べる．
